# Safety-first Risk Measures

Developed based on the book Handbook of Finance, the 2 safety measures that were requested to be included in the financial dashboard were the Minimax (MM) and Value at Risk (VaR) measures. These measures are computed to help understand the value that an investor could lose out on if they were to invest in the stock. While these may be short sided measures, it could help an investor understand what to expect in the short term to take on more risk in the long term.

We first start by importing some packages that allow us to utilize the MongoDB Altas cloud database and access a financial packages that simplifies calculating the two well know safety measures. These pages are also installed when the requirements files is run.

In [1]:
# pip install pyfolio
# pip install pymongo
# pip install "pymongo[srv]"

In [2]:
%cd ../

/Users/tiareina/PycharmProjects/pythonProject


In [15]:
from pprint import pprint
from typing import Union
from src.data import sec
from src.data.yahoo import retrieve_company_stock_price_from_mongo

import pymongo
import configparser
import json
import os

import yfinance as yf
import datetime as dt
import pandas as pd
import numpy as np
import pyfolio as pf

ModuleNotFoundError: No module named 'ratelimit'

## Retrieving the Data
Joshua R has spun up a MongoDB Atlas instance and provided the team with credentials. Using the Yahoo data, I will be able to perform VAR and MM with the pyfolio package. I'll first log into the database to develop an understand of the contents.

In [28]:
config = configparser.ConfigParser()
config.read('src/config.ini')
mongo_config = config['MONGO']
mongo_connection = "mongodb+srv://" + mongo_config['User'] + ":" + mongo_config['Password'] + "@" + mongo_config['Address']
myclient = pymongo.MongoClient(mongo_connection)

In [23]:
mydb = myclient["market_shopper"]
sub_col = mydb["sec_sub"]
num_col = mydb["sec_num"]
tag_col = mydb["sec_tag"]
pre_col = mydb["sec_pre"]
sec_col = mydb["sec"]
yahoo_col = mydb["yahoo"]

In [35]:
yahoo_col['SPY'].find_one()

OperationFailure: bad auth : Authentication failed., full error: {'ok': 0, 'errmsg': 'bad auth : Authentication failed.', 'code': 8000, 'codeName': 'AtlasError'}

In [18]:
def retrieve_close(ticker:'str',start:dt.datetime=dt.date.today(),end:dt.datetime=dt.date.today()):
    detail = {'ticker':ticker}
    rows = yahoo_col.find_one(detail)
    if rows is None:
        return None
    else:
        rows = rows['stock_price']
        close = [(row['Date'],row['Close']) for row in rows 
                 if row['Date'].date() >= start and row['Date'].date() <= end]
    return pd.DataFrame(close, columns=['Date','Close'])

def get_noncumulative_returns(df:pd.DataFrame):
    return df['Close'].pct_change()

def get_kpis(portfolio_returns:pd.DataFrame,baseline_returns:pd.DataFrame):
    return pf.show_perf_stats(portfolio_returns,baseline_returns)

In [19]:
def add_portfolio_weights(portfolio:dict[str,dict]):
    """
    Calculates the total of a portfolio (dict(str:dict))
    then computes the weight for each stock within the
    portfolio. Stock weights will be used in VaR.
    
    Return:
        portfolio: dict
    """
    portfolio_stocks = list(portfolio.keys())
    portfolio_value = 0
    for stock in portfolio_stocks:
        if 'value' in portfolio[stock]:
            portfolio_value += portfolio[stock]['value']

    for stock in portfolio_stocks:
        if 'value' in portfolio[stock]:
            stock_weight = portfolio[stock]['value'] / portfolio_value
            portfolio[stock]['weight'] = round(stock_weight, 3)

    assert (np.isclose([round(sum([portfolio[stock]['weight'] for stock in portfolio_stocks 
                                   if 'weight' in portfolio[stock]]), 2)], [1.0]) == True).all()
    
    return portfolio


def calculate_VaR(portfolio=dict[str,dict],
                  index:str='SPY',
                  start:dt.datetime=dt.date.today(),
                  end:dt.datetime=dt.date.today()):
    """
    Calculates the total of a portfolio (dict(key:dict))
    and computes the weight for each stock within the
    portfolio. Stock weights will be used in VaR.
    
    Return:
        Value at Risk: value or dataframe
    """
    portfolio_stocks = list(portfolio.keys())
    returns = pd.DataFrame()
    weights = []
    
    for stock in portfolio_stocks:
        s = portfolio[stock]
        close = retrieve_close(stock,start,end)
        if close is not None:
            ret = get_noncumulative_returns(close)
            returns[stock] = get_kpis(ret)
            weights.append(s['weight'])
    return returns, weights



In [20]:
input_data = '{"BTC": {"name": "BTC", "type": "cash", "quantity": 0.00293644, "value": 115.57268}, "MIPTX": {"name": "MIPTX", "type": "mutual fund", "quantity": 23.567, "value": 636.309}, "ACHN": {"name": "ACHN", "type": "equity", "quantity": 1.0, "value": 2.11}, "CAMYX": {"name": "CAMYX", "type": "mutual fund", "quantity": 75.75, "value": 1855.875}, "DBLTX": {"name": "DBLTX", "type": "mutual fund", "quantity": 2.0, "value": 20.84}, "PRFDX": {"name": "PRFDX", "type": "mutual fund"}, "EWZ": {"name": "EWZ", "type": "etf", "quantity": 5.0, "value": 210.75}, "null": {"name": null, "type": "mutual fund", "quantity": 21.5, "value": 430.0}, "SBSI": {"name": "SBSI", "type": "equity", "quantity": 213.0, "value": 7397.49}, "NHX105509": {"name": "NHX105509", "type": "etf", "quantity": 100.05, "value": 1373.6865}}'
input_data = json.loads(input_data)
p = add_portfolio_weights(input_data)
calculate_VaR(p)


OperationFailure: bad auth : Authentication failed., full error: {'ok': 0, 'errmsg': 'bad auth : Authentication failed.', 'code': 8000, 'codeName': 'AtlasError'}

## VAR
We will compare the noncumulative results for a given ti

In [26]:
# TODO: Turn into unit test.
# This test emulates someone with a portfolio of just one stock
# The performance is measured against that of SPY, which is a 
# standard baseline for the US Market.
bac = retrieve_data('BAC', '1mo')
spy = retrieve_data('SPY', '1mo')

bac_returns = get_noncumulative_returns(bac)
spy_returns = get_noncumulative_returns(spy)

define_value_at_risk(bac_returns,spy_returns)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Start date,2022-09-15
End date,2022-10-14
Total months,1
,Backtest
Annual return,-62.2%
Cumulative returns,-8.1%
Annual volatility,38.3%
Sharpe ratio,-2.47
Calmar ratio,-4.39
Stability,0.45
Max drawdown,-14.2%


## MiniMax

Explain minimax calculation

## Visualizing with pyChart.js